In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nltk.tokenize import WordPunctTokenizer
import gensim
from gensim.models import KeyedVectors
from IPython.display import clear_output
from collections import Counter
from tqdm import tqdm
import seaborn as sns
import nltk
import json

%matplotlib inline

In [2]:
from pymystem3 import Mystem
from string import punctuation

In [3]:
def read_queries_with_lemmatization(path):
    f = open(path)
    queries = []
    tags = []
    mystem = Mystem()
    tokenizer = WordPunctTokenizer()
    
    for line in f:
        tmp = []
        tmp_ = []
        text = tokenizer.tokenize(line.lower())
        text_tagged = nltk.pos_tag(text, lang='rus')
        
        for i, q in enumerate(text):
            if not np.all(np.any(np.array(list(q)).reshape(-1, 1) == np.array(list(punctuation)).reshape(1, -1), axis=1)):
                q_ = mystem.lemmatize(q)
                tmp.append("".join(q_).split()[0])
                tmp_.append(text_tagged[i][1])
        queries.append(tmp)
        tags.append(tmp_)
    f.close()
    return (queries, tags)

In [4]:
path = 'data/requests.uniq.train'
train_lem = read_queries_with_lemmatization(path)
path = 'data/requests.uniq.test'
test_lem = read_queries_with_lemmatization(path)
train_lem[0][:5], train_lem[1][:5], test_lem[0][:5], test_lem[1][:5], len(train_lem[0]), len(test_lem[0])

([['сибирский', 'сеть', 'личный', 'кабинет', 'бердск'],
  ['1', 'сантим', 'алжир', '1964'],
  ['река', 'колыма', 'на', 'карта', 'россия'],
  ['ноофен', 'для', 'какой', 'болезнь'],
  ['маус', 'хаус', 'спб']],
 [['A=pl', 'S', 'A=m', 'S', 'S'],
  ['NUM=ciph', 'V', 'S', 'NUM=ciph'],
  ['S', 'S', 'PR', 'S', 'S'],
  ['V', 'PR', 'A-PRO=pl', 'A=f'],
  ['NONLEX', 'NONLEX', 'NONLEX']],
 [['сбербанк', 'в', 'кунцево', 'плаза'],
  ['торт', 'дикий', 'вишня'],
  ['тася', 'кривун', 'танец', 'на', 'тнт'],
  ['рбт', 'ру'],
  ['toplü', 'vay', 'sexx']],
 [['V', 'PR', 'S', 'S'],
  ['S', 'A=f', 'S'],
  ['S', 'S', 'S', 'PR', 'S'],
  ['V', 'S'],
  ['NONLEX', 'NONLEX', 'NONLEX']],
 51353,
 21174)

In [5]:
count_words = Counter()

for d in [train_lem[0]]:
    for q in d:
        for word in q:
            count_words[word] += 1
        
freq, counts = np.unique(np.array(list(count_words.values())), return_counts=True) 
p = counts * freq 
p = p / p.sum()
p = np.cumsum(p)
freq[:10], p[:10]

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([0.12185658, 0.16966756, 0.20326518, 0.23018925, 0.2515358 ,
        0.27062838, 0.28792218, 0.30358516, 0.31885922, 0.32964299]))

In [6]:
tmp = np.unique(np.hstack(train_lem[1]))
tmp1 = np.unique(np.hstack(test_lem[1]))
tmp = np.hstack([tmp, tmp1])
tmp = np.unique(tmp)
tmp, len(tmp)

(array(['A', 'A-PRO', 'A-PRO=f', 'A-PRO=m', 'A-PRO=n', 'A-PRO=pl',
        'A-PRO=sg', 'A=brev', 'A=comp', 'A=comp2', 'A=f', 'A=m', 'A=n',
        'A=pl', 'A=sg', 'ADV', 'ADV-PRO', 'ADV-PRO=abbr', 'ADV-PRO=comp',
        'ADV-PRO=distort', 'ADV=abbr', 'ADV=comp', 'ADV=comp2',
        'ANUM=ciph', 'ANUM=f', 'ANUM=m', 'ANUM=n', 'ANUM=pl', 'ANUM=sg',
        'CONJ', 'INIT=abbr', 'INTJ', 'INTJ=distort', 'NONLEX',
        'NONLEX=abbr', 'NUM', 'NUM=acc', 'NUM=ciph', 'NUM=comp', 'NUM=dat',
        'NUM=f', 'NUM=gen', 'NUM=ins', 'NUM=loc', 'NUM=m', 'NUM=n',
        'NUM=nom', 'PARENTH', 'PART', 'PR', 'PRAEDIC', 'PRAEDIC-PRO',
        'PRAEDIC=comp', 'S', 'S-PRO', 'S-PRO=acc', 'S-PRO=dat',
        'S-PRO=gen', 'S-PRO=ins', 'S-PRO=loc', 'S-PRO=n=sg', 'S-PRO=pl',
        'S=m', 'V'], dtype='<U32'), 64)

In [7]:
tags = tmp
tags_to_ind = {}
ind = 0
for t in tags:
    tags_to_ind[t] = ind
    ind += 1

## Сетки

* Просто добавляем тег как фичу

In [8]:
emb_2 = KeyedVectors.load_word2vec_format("ft_native_300_ru_wiki_lenta_lower_case.vec")

In [9]:
import torch, torch.nn as nn
import torch.nn.functional as F

In [10]:
def calculate_n_tokens(emb):
    n_tokens = 0
    for word in count_words.keys():
        if word in emb.vocab and count_words[word] >= 3:
            n_tokens += 1
    return n_tokens + 1

In [11]:
def transform_to_features(emb, emb_size, ind_to_word, batch_x, batch_x_tags):
    to_emb = np.zeros((len(batch_x), len(batch_x[0])+1, emb_size + len(tags)))
    for i in range(len(batch_x)):
        to_emb[i][0] = np.ones(emb_size + len(tags))
        for j in range(len(batch_x[i])):
            if batch_x[i][j] != pad_id:
                to_emb[i][j+1][:emb_size] = emb[ind_to_word[batch_x[i][j]]]
                if batch_x_tags[i][j] >= 0:
                    to_emb[i][j+1][emb_size + batch_x_tags[i][j]] = 1
    return to_emb

In [12]:
class Net(nn.Module):
    def __init__(self, emb, ind_to_word, emb_size=300, lstm_units=256):
        super(self.__class__, self).__init__()
        n_tokens = calculate_n_tokens(emb)
        self.lstm = nn.LSTM(emb_size + len(tags), lstm_units, batch_first=True)
        self.logits = nn.Linear(lstm_units, n_tokens) 
        self.emb = emb
        self.emb_size = emb_size
        self.ind_to_word = ind_to_word
        
    def forward(self, batch_x, batch_x_tags):
        input_emb = transform_to_features(self.emb, self.emb_size, self.ind_to_word, batch_x, batch_x_tags)
        input_emb = torch.tensor(input_emb, dtype=torch.float32)
        lstm_out = self.lstm(input_emb)
        logits = self.logits(lstm_out[0])
        
        return logits

In [13]:
pad = '#PAD#'
pad_id = 0

def construct_vocab(emb, count_words):
    word_to_ind = dict()
    word_to_ind['#PAD#'] = 0
    ind_to_word = ['#PAD#', ]
    
    count = 1
    for word in count_words.keys():
        if count_words[word] >= 3 and word in emb.vocab:
            ind_to_word.append(word)
            word_to_ind[word] = count
            count += 1
    return ind_to_word, word_to_ind


def as_matrix(sequences, tags, word_to_ind, max_len=None):
    """ Convert a list of tokens into a matrix with padding """
    max_len = max_len or max(map(len,sequences))
    
    matrix = np.zeros((2, len(sequences), max_len), dtype=int)
    for i, seq in enumerate(sequences):
        for j, word in enumerate(seq[:max_len]):
            if word in word_to_ind.keys():
                matrix[0][i][j] = word_to_ind[word]
                matrix[1][i][j] = tags_to_ind[tags[i][j]]
            else:
                matrix[0][i][j] = pad_id
                matrix[1][i][j] = -1
        for j in range(max_len, len(seq)):
            matrix[0][i][j] = pad_id
            matrix[1][i][j] = -1
    
    return matrix

In [14]:
def compute_loss(network, batch):
    """
    use scalar crossentropy loss (neg llh) loss 
    """
    batch_x = batch[0]
    batch_tags = batch[1]
    batch_x = np.array(batch_x)
    batch_tags = np.array(batch_tags)

    batch_x_inp = batch_x[:, :-1]
    batch_x_next = batch_x[:, 1:]
    batch_tags_inp = batch_tags[:, :-1]
    batch_tags_next = batch_tags[:, 1:]
    
    logits_for_next = network.forward(batch_x_inp, batch_tags_inp)
    logits_for_next = logits_for_next[:, 1:]
    
    answers = torch.argmax(logits_for_next, dim=-1).numpy()
    logits_for_next = logits_for_next.contiguous()
    logits_for_next = logits_for_next.view(-1, logits_for_next.shape[-1])
    
    accr = np.array([answers == batch_x_next]) * np.array([answers != pad_id])
    accr = accr.sum()
    to_div = np.sum(np.array([batch_x_next != pad_id]))
    batch_x_next = torch.tensor(batch_x_next, dtype=torch.int64)
    batch_x_next = batch_x_next.view(-1)
    
    loss = F.cross_entropy(logits_for_next, batch_x_next, ignore_index=pad_id, reduction='mean')
    
    
    return loss, accr, to_div

In [15]:
from random import choice

def generate_batch(train, batch_size, word_to_ind, max_len=None):
    random_x = np.random.randint(0, len(train[0]), size=batch_size)
    batch_x = []
    batch_tags = []
    for x in random_x:
        batch_x.append(train[0][x])
        batch_tags.append(train[1][x])
    return as_matrix(batch_x, batch_tags, word_to_ind, max_len)

In [16]:
batch_size = 64 
n_epochs = 20 
n_batches_per_epoch = 400  
n_validation_batches = 160

In [85]:
from torch.optim import Adam
from tqdm import tqdm

ind_to_word, word_to_ind = construct_vocab(emb_2, count_words)
network = Net(emb_2, ind_to_word)
opt = Adam(network.parameters())

train_loss, val_loss, train_accr, val_accr = [], [], [], []

for epoch in range(n_epochs):
    train_loss_=0
    train_accr_=0
    to_div = 0
    network.train(True)
    for _ in tqdm(range(n_batches_per_epoch)):
        
        loss_t, accr_t, to_div_t = compute_loss(network, generate_batch(train_lem, batch_size, word_to_ind))
        
        loss_t.backward()
        opt.step()
        opt.zero_grad()
        
        train_loss_ += loss_t.item()
        train_accr_ += accr_t.item()
        to_div += to_div_t
        
    train_loss_ /= n_batches_per_epoch
    #train_accr_ /= n_batches_per_epoch
    train_accr_ /= to_div
    
    val_loss_=0
    val_accr_=0
    to_div = 0
    network.train(False)
    for _ in range(n_validation_batches):
        loss_t, accr_t, to_div_t = compute_loss(network, generate_batch(test_lem, batch_size, word_to_ind))
        
        val_loss_ += loss_t.item()
        val_accr_ += accr_t.item()
        to_div += to_div_t
        
    val_loss_ /= n_validation_batches
    #val_accr_ /= n_validation_batches
    val_accr_ /= to_div
    
    train_loss.append(train_loss_)
    val_loss.append(val_loss_)
    train_accr.append(train_accr_)
    val_accr.append(val_accr_)
    
    print('\nEpoch: {}, train loss: {}, val loss: {}'.format(epoch, train_loss_, val_loss_))
    print('\nEpoch: {}, train accr: {}, val accr: {}'.format(epoch, train_accr_, val_accr_))

print("Finished!")

  0%|          | 0/400 [00:00<?, ?it/s]


Epoch: 0, train loss: 7.415038217306137, val loss: 7.029473960399628

Epoch: 0, train accr: 0.06012615392945126, val accr: 0.08210409128861676


  0%|          | 0/400 [00:00<?, ?it/s]


Epoch: 1, train loss: 6.588179706335068, val loss: 6.555705967545509

Epoch: 1, train accr: 0.10989722424536699, val accr: 0.10720432751413818


  0%|          | 0/400 [00:00<?, ?it/s]

...


Epoch: 17, train loss: 3.459779422879219, val loss: 6.2179121434688565

Epoch: 17, train accr: 0.3693340342080789, val accr: 0.16126997476871321


  0%|          | 0/400 [00:00<?, ?it/s]


Epoch: 18, train loss: 3.383422926068306, val loss: 6.212906065583229

Epoch: 18, train accr: 0.38028302014056575, val accr: 0.16483516483516483


100%|██████████| 400/400 [02:51<00:00,  2.35it/s]



Epoch: 19, train loss: 3.2591949808597565, val loss: 6.271716690063476

Epoch: 19, train accr: 0.3971612212104981, val accr: 0.15462953340953964
Finished!


In [86]:
torch.save(network.state_dict(), 'add_pos_tagging_as_feature.pwf')

In [97]:
def compute_accr(network, batch):
    batch_x = batch[0]
    batch_tag = batch[1]
    
    batch_x = np.array(batch_x)
    batch_tag = np.array(batch_tag)
    batch_x_inp = batch_x[:, :-1]
    batch_x_next = batch_x[:, 1:]
    batch_tag_inp = batch_tag[:, :-1]
    batch_tag_next = batch_tag[:, 1:]
    
    logits_for_next = network.forward(batch_x_inp, batch_tag_inp)
    logits_for_next = logits_for_next[:, 1:]
    
    answers = torch.argmax(logits_for_next, dim=-1).numpy()
    
    accr = np.array([answers == batch_x_next]) * np.array([answers != pad_id])
    accr = accr[0]
    accr = accr.sum(axis=0)
    to_divide = np.array([batch_x_next != pad_id])[0].sum(axis=0)
    
    return accr, to_divide

def get_batch(data, left, right, batch_size, word_to_ind, max_len=None):
    slice_x = np.arange(left, right, 1)
    batch_x = []
    batch_tag = []
    for x in slice_x:
        batch_x.append(data[0][x])
        batch_tag.append(data[1][x])
    return as_matrix(batch_x, batch_tag, word_to_ind, max_len)

def try_lengthes(data):
    accr = np.zeros(np.max(list(map(len, data[0]))))
    to_div = np.zeros(np.max(list(map(len, data[0]))))
    
    for _ in tqdm_notebook(range(0, len(data[0])-batch_size, batch_size)):
        accr_t, div_t = compute_accr(network, get_batch(data, _, _+batch_size, batch_size, word_to_ind))
        accr[:len(accr_t)] += accr_t
        to_div[:len(div_t)] += div_t
    eps = 1
    return accr / (to_div + eps), accr.sum() / to_div.sum()

In [30]:
def approximate_pad(data):
    to_pad = 0
    all_ = 0
    for x in data:
        for word in x:
            all_ += 1
            if word not in word_to_ind.keys():
                to_pad += 1
    return to_pad / all_

In [99]:
from tqdm import tqdm_notebook

on_train, all_accr = try_lengthes(train_lem)
on_train[:10], all_accr

(array([0.22534229, 0.3419334 , 0.46927353, 0.51727253, 0.55219961,
        0.5572743 , 0.57848837, 0.58447489, 0.56936226, 0.54239257]),
 0.4007140359841863)

In [100]:
on_test, all_accr = try_lengthes(test_lem)
on_test[:10], all_accr

(array([0.13774834, 0.14425538, 0.16985902, 0.1910342 , 0.18946509,
        0.19265442, 0.19807281, 0.18886861, 0.16057234, 0.11239193]),
 0.1608981155306269)

* попробуем учить предсказание следующего тега

In [17]:
class NetWithTags(nn.Module):
    def __init__(self, emb, ind_to_word, emb_size=300, lstm_units=256):
        super(self.__class__, self).__init__()
        n_tokens = calculate_n_tokens(emb)
        self.lstm = nn.LSTM(emb_size + len(tags), lstm_units, batch_first=True)
        self.logits = nn.Linear(lstm_units, n_tokens) 
        self.logits_tags = nn.Linear(lstm_units, len(tags))
        self.emb = emb
        self.emb_size = emb_size
        self.ind_to_word = ind_to_word
        
    def forward(self, batch_x, batch_x_tags):
        input_emb = transform_to_features(self.emb, self.emb_size, self.ind_to_word, batch_x, batch_x_tags)
        input_emb = torch.tensor(input_emb, dtype=torch.float32)
        lstm_out = self.lstm(input_emb)
        logits = self.logits(lstm_out[0])
        logits_tags = self.logits_tags(lstm_out[0])
        return logits, logits_tags

In [18]:
def compute_loss(network, batch):
    """
    use scalar crossentropy loss (neg llh) loss 
    """
    batch_x = batch[0]
    batch_tags = batch[1]
    batch_x = np.array(batch_x)
    batch_tags = np.array(batch_tags)

    batch_x_inp = batch_x[:, :-1]
    batch_x_next = batch_x[:, 1:]
    batch_tags_inp = batch_tags[:, :-1]
    batch_tags_next = batch_tags[:, 1:]
    
    logits_for_next, logits_tags_for_next = network.forward(batch_x_inp, batch_tags_inp)
    logits_for_next = logits_for_next[:, 1:]
    logits_tags_for_next = logits_tags_for_next[:, 1:]
    
    answers = torch.argmax(logits_for_next, dim=-1).numpy()
    tags = torch.argmax(logits_tags_for_next, dim=-1).numpy()
    logits_for_next = logits_for_next.contiguous()
    logits_for_next = logits_for_next.view(-1, logits_for_next.shape[-1])
    logits_tags_for_next = logits_tags_for_next.contiguous()
    logits_tags_for_next = logits_tags_for_next.view(-1, logits_tags_for_next.shape[-1])
    
    accr = np.array([answers == batch_x_next]) * np.array([answers != pad_id])
    accr = accr.sum()
    accr_tags = np.array([tags == batch_tags_next]) * np.array([batch_x_next != pad_id])
    accr_tags = accr_tags.sum()
    to_div = np.sum(np.array([batch_x_next != pad_id]))
    
    batch_x_next = torch.tensor(batch_x_next, dtype=torch.int64)
    batch_x_next = batch_x_next.view(-1)
    batch_tags_next = torch.tensor(batch_tags_next, dtype=torch.int64)
    batch_tags_next = batch_tags_next.view(-1)
    
    loss = F.cross_entropy(logits_for_next, batch_x_next, ignore_index=pad_id, reduction='mean')
    loss += F.cross_entropy(logits_tags_for_next, batch_tags_next, ignore_index=-1, reduction='mean')
    
    return loss, accr, accr_tags, to_div

In [19]:
from torch.optim import Adam
from tqdm import tqdm

ind_to_word, word_to_ind = construct_vocab(emb_2, count_words)
network = NetWithTags(emb_2, ind_to_word)
opt = Adam(network.parameters())

train_loss, val_loss, train_accr, val_accr, train_tags_accr, val_tags_accr = [], [], [], [], [], []

for epoch in range(n_epochs):
    train_loss_=0
    train_accr_=0
    train_accr_tags_=0
    to_div = 0
    network.train(True)
    for _ in tqdm(range(n_batches_per_epoch)):
        
        loss_t, accr_t, accr_tags_t, to_div_t = compute_loss(network, generate_batch(train_lem, batch_size, word_to_ind))
        
        loss_t.backward()
        opt.step()
        opt.zero_grad()
        
        train_loss_ += loss_t.item()
        train_accr_ += accr_t.item()
        train_accr_tags_ += accr_tags_t.item()
        to_div += to_div_t
        
    train_loss_ /= n_batches_per_epoch
    #train_accr_ /= n_batches_per_epoch
    train_accr_ /= to_div
    train_accr_tags_ /= to_div
    
    val_loss_=0
    val_accr_=0
    val_accr_tags_=0
    to_div = 0
    network.train(False)
    for _ in range(n_validation_batches):
        loss_t, accr_t, accr_tags_t, to_div_t = compute_loss(network, generate_batch(test_lem, batch_size, word_to_ind))
        
        val_loss_ += loss_t.item()
        val_accr_ += accr_t.item()
        val_accr_tags_ += accr_tags_t.item()
        to_div += to_div_t
        
    val_loss_ /= n_validation_batches
    #val_accr_ /= n_validation_batches
    val_accr_ /= to_div
    val_accr_tags_ /= to_div
    
    train_loss.append(train_loss_)
    val_loss.append(val_loss_)
    train_accr.append(train_accr_)
    val_accr.append(val_accr_)
    train_tags_accr.append(train_accr_tags_)
    val_tags_accr.append(val_accr_tags_)
    
    print('\nEpoch: {}, train loss: {}, val loss: {}'.format(epoch, train_loss_, val_loss_))
    print('\nEpoch: {}, train accr: {}, val accr: {}'.format(epoch, train_accr_, val_accr_))
    print('\nEpoch: {}, train accr tags: {}, val accr tags: {}'.format(epoch, train_accr_tags_, val_accr_tags_))

print("Finished!")

  0%|          | 0/400 [00:00<?, ?it/s]


Epoch: 0, train loss: 8.267957304716111, val loss: 7.728692108392716

Epoch: 0, train accr: 0.05421031152224187, val accr: 0.07534610345926425

Epoch: 0, train accr tags: 0.3698000272071827, val accr tags: 0.4069680982898417


  0%|          | 0/400 [00:00<?, ?it/s]


Epoch: 1, train loss: 7.374649313688278, val loss: 7.271440061926842

Epoch: 1, train accr: 0.10069555484489927, val accr: 0.09989834904833678

Epoch: 1, train accr tags: 0.43771985929005436, val accr tags: 0.43352378281748394


  0%|          | 0/400 [00:00<?, ?it/s]

...


Epoch: 17, train loss: 4.119367979168892, val loss: 6.804924938082695

Epoch: 17, train accr: 0.34934791981030394, val accr: 0.16087094387485468

Epoch: 17, train accr tags: 0.5083800388014659, val accr tags: 0.44833174787725044


  0%|          | 0/400 [00:00<?, ?it/s]


Epoch: 18, train loss: 4.033529533743859, val loss: 6.821166551113128

Epoch: 18, train accr: 0.365685019206146, val accr: 0.16335493160547157

Epoch: 18, train accr tags: 0.5113282566210661, val accr tags: 0.4712023038156947


100%|██████████| 400/400 [02:55<00:00,  2.72it/s]



Epoch: 19, train loss: 3.9156561106443406, val loss: 6.831090602278709

Epoch: 19, train accr: 0.380852176010937, val accr: 0.16100197851893724

Epoch: 19, train accr tags: 0.5127263466873969, val accr tags: 0.4583451102317694
Finished!


In [20]:
torch.save(network.state_dict(), 'add_pos_tagging_as_feature_and_in_loss.pwf')

In [23]:
def compute_accr(network, batch):
    batch_x = batch[0]
    batch_tag = batch[1]
    
    batch_x = np.array(batch_x)
    batch_tag = np.array(batch_tag)
    batch_x_inp = batch_x[:, :-1]
    batch_x_next = batch_x[:, 1:]
    batch_tag_inp = batch_tag[:, :-1]
    batch_tag_next = batch_tag[:, 1:]
    
    logits_for_next, logits_tags_for_next = network.forward(batch_x_inp, batch_tag_inp)
    logits_for_next = logits_for_next[:, 1:]
    logits_tags_for_next = logits_tags_for_next[:, 1:]
    
    answers = torch.argmax(logits_for_next, dim=-1).numpy()
    tags = torch.argmax(logits_tags_for_next, dim=-1).numpy()
    
    accr = np.array([answers == batch_x_next]) * np.array([answers != pad_id])
    accr = accr[0]
    accr = accr.sum(axis=0)
    accr_tags = np.array([tags == batch_tag_next]) * np.array([batch_x_next != pad_id])
    accr_tags = accr_tags[0]
    accr_tags = accr_tags.sum(axis=0)
    to_divide = np.array([batch_x_next != pad_id])[0].sum(axis=0)
    
    return accr, accr_tags, to_divide

def get_batch(data, left, right, batch_size, word_to_ind, max_len=None):
    slice_x = np.arange(left, right, 1)
    batch_x = []
    batch_tag = []
    for x in slice_x:
        batch_x.append(data[0][x])
        batch_tag.append(data[1][x])
    return as_matrix(batch_x, batch_tag, word_to_ind, max_len)

def try_lengthes(data):
    accr = np.zeros(np.max(list(map(len, data[0]))))
    accr_tags = np.zeros(np.max(list(map(len, data[0]))))
    to_div = np.zeros(np.max(list(map(len, data[0]))))
    
    for _ in tqdm_notebook(range(0, len(data[0])-batch_size, batch_size)):
        accr_t, accr_tags_t, div_t = compute_accr(network, get_batch(data, _, _+batch_size, batch_size, word_to_ind))
        accr[:len(accr_t)] += accr_t
        to_div[:len(div_t)] += div_t
        accr_tags[:len(accr_tags_t)] += accr_tags_t
    eps = 1
    return accr / (to_div + eps), accr_tags / (to_div + eps), accr.sum() / to_div.sum(), accr_tags.sum() / to_div.sum()

In [25]:
from tqdm import tqdm_notebook

on_train, on_train_tags, all_accr, all_accr_tags = try_lengthes(train_lem)
on_train[:10], on_train_tags[:10], all_accr, all_accr_tags

(array([0.219969  , 0.32748882, 0.45278524, 0.49261267, 0.53258372,
        0.52887052, 0.55344365, 0.54452055, 0.53320184, 0.49361208]),
 array([0.60958409, 0.51012003, 0.48772832, 0.4594494 , 0.42769206,
        0.40512068, 0.40317531, 0.40905632, 0.40368179, 0.40998839]),
 0.384947422209074,
 0.5055805072210419)

In [26]:
on_test, on_test_tags, all_accr, all_accr_tags = try_lengthes(test_lem)
on_test[:10], on_test_tags[:10], all_accr, all_accr_tags

(array([0.14      , 0.14794459, 0.17433155, 0.18912658, 0.19334422,
        0.20400668, 0.20396146, 0.1979927 , 0.16216216, 0.11239193]),
 array([0.58516556, 0.45859057, 0.41915411, 0.39801063, 0.36504696,
        0.35358932, 0.32601713, 0.34032847, 0.3608903 , 0.32564841]),
 0.16418140155272068,
 0.45334997776941754)

* посмотрим насколько знание тэга может улучшить биграммное предсказание

In [28]:
cnt = dict()
for i, line in tqdm_notebook(enumerate(train_lem[0])):
    prefix = ""
    for j, word in enumerate(line):
        if prefix not in cnt.keys():
            cnt[prefix] = Counter()
        cnt[prefix][word] += 1
        prefix = word

accr = np.zeros(np.max(list(map(len, test_lem[0]))))
to_div = np.zeros_like(accr)
for i, line in tqdm_notebook(enumerate(test_lem[0])):
    prefix = ""
    for j, word in enumerate(line):
        to_div[j] += 1
        if prefix in cnt.keys() and cnt[prefix].most_common(1)[0][0] == word:
            accr[j] += 1
        prefix = word

In [29]:
accr / to_div * 100

array([ 2.91422634, 11.6593975 , 10.16124125, 12.48063911, 12.96511628,
       12.91005291, 14.01091013, 14.58625526, 12.5399361 , 12.82051282,
        8.27250608,  7.239819  ,  6.81818182,  4.87804878,  3.33333333,
        0.        ,  3.03030303,  3.44827586,  4.34782609,  5.55555556,
        6.66666667,  7.14285714,  7.69230769,  0.        ,  8.33333333,
        0.        ,  0.        ,  0.        ,  0.        , 12.5       ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , 25.        ,  0.        ,  0.        ,
        0.        ,  0.        , 33.33333333,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ])

In [34]:
cnt = dict()
for i, line in tqdm_notebook(enumerate(train_lem[0])):
    prefix = ""
    prefix_tag = ""
    for j, word in enumerate(line):
        if prefix not in cnt.keys():
            cnt[prefix] = dict()
        if prefix_tag not in cnt[prefix].keys():
            cnt[prefix][prefix_tag] = Counter()
        cnt[prefix][prefix_tag][word] += 1
        prefix = word
        prefix_tag = train_lem[1][i][j]

accr = np.zeros(np.max(list(map(len, test_lem[0]))))
to_div = np.zeros_like(accr)
for i, line in tqdm_notebook(enumerate(test_lem[0])):
    prefix = ""
    prefix_tag = ""
    for j, word in enumerate(line):
        to_div[j] += 1
        if prefix in cnt.keys() and prefix_tag in cnt[prefix].keys() and cnt[prefix][prefix_tag].most_common(1)[0][0] == word:
            accr[j] += 1
        prefix = word
        prefix_tag = test_lem[1][i][j]

In [35]:
accr / to_div * 100

array([ 2.91422634, 12.13611928, 10.31335564, 12.71704573, 13.46511628,
       13.22751323, 14.38415159, 14.53950444, 12.69968051, 13.09041835,
        8.27250608,  7.69230769,  6.81818182,  3.65853659,  3.33333333,
        0.        ,  3.03030303,  3.44827586,  4.34782609,  5.55555556,
        6.66666667,  7.14285714,  7.69230769,  0.        ,  8.33333333,
        0.        ,  0.        ,  0.        ,  0.        , 25.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , 50.        ,  0.        ,  0.        ,
        0.        ,  0.        , 33.33333333,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ])

* попробуем учить промежуточное представление тег

In [75]:
class NetWithInnerTags(nn.Module):
    def __init__(self, emb, ind_to_word, emb_size=300, lstm_units=256):
        super(self.__class__, self).__init__()
        n_tokens = calculate_n_tokens(emb)
        self.lstm = nn.LSTM(emb_size + len(tags), lstm_units, batch_first=True)
        self.logits_tags = nn.Linear(lstm_units, len(tags))
        self.logits = nn.Linear(lstm_units + len(tags), n_tokens) 
        self.emb = emb
        self.emb_size = emb_size
        self.ind_to_word = ind_to_word
        
    def forward(self, batch_x, batch_x_tags):
        input_emb = transform_to_features(self.emb, self.emb_size, self.ind_to_word, batch_x, batch_x_tags)
        input_emb = torch.tensor(input_emb, dtype=torch.float32)
        lstm_out = self.lstm(input_emb)
        logits_tags = self.logits_tags(lstm_out[0])
        #print(lstm_out[0].shape)
        #print(logits_tags.shape)
        inner = torch.cat([lstm_out[0], logits_tags], dim=2)
        #print(inner.shape)
        logits = self.logits(inner)
        return logits, logits_tags

In [76]:
from torch.optim import Adam
from tqdm import tqdm

ind_to_word, word_to_ind = construct_vocab(emb_2, count_words)
network = NetWithInnerTags(emb_2, ind_to_word)
opt = Adam(network.parameters())

train_loss, val_loss, train_accr, val_accr, train_tags_accr, val_tags_accr = [], [], [], [], [], []

for epoch in range(n_epochs):
    train_loss_=0
    train_accr_=0
    train_accr_tags_=0
    to_div = 0
    network.train(True)
    for _ in tqdm(range(n_batches_per_epoch)):
        
        loss_t, accr_t, accr_tags_t, to_div_t = compute_loss(network, generate_batch(train_lem, batch_size, word_to_ind))
        
        loss_t.backward()
        opt.step()
        opt.zero_grad()
        
        train_loss_ += loss_t.item()
        train_accr_ += accr_t.item()
        train_accr_tags_ += accr_tags_t.item()
        to_div += to_div_t
        
    train_loss_ /= n_batches_per_epoch
    #train_accr_ /= n_batches_per_epoch
    train_accr_ /= to_div
    train_accr_tags_ /= to_div
    
    val_loss_=0
    val_accr_=0
    val_accr_tags_=0
    to_div = 0
    network.train(False)
    for _ in range(n_validation_batches):
        loss_t, accr_t, accr_tags_t, to_div_t = compute_loss(network, generate_batch(test_lem, batch_size, word_to_ind))
        
        val_loss_ += loss_t.item()
        val_accr_ += accr_t.item()
        val_accr_tags_ += accr_tags_t.item()
        to_div += to_div_t
        
    val_loss_ /= n_validation_batches
    #val_accr_ /= n_validation_batches
    val_accr_ /= to_div
    val_accr_tags_ /= to_div
    
    train_loss.append(train_loss_)
    val_loss.append(val_loss_)
    train_accr.append(train_accr_)
    val_accr.append(val_accr_)
    train_tags_accr.append(train_accr_tags_)
    val_tags_accr.append(val_accr_tags_)
    
    print('\nEpoch: {}, train loss: {}, val loss: {}'.format(epoch, train_loss_, val_loss_))
    print('\nEpoch: {}, train accr: {}, val accr: {}'.format(epoch, train_accr_, val_accr_))
    print('\nEpoch: {}, train accr tags: {}, val accr tags: {}'.format(epoch, train_accr_tags_, val_accr_tags_))

print("Finished!")

  0%|          | 0/400 [00:00<?, ?it/s]


Epoch: 0, train loss: 8.303791218996048, val loss: 7.757710286974907

Epoch: 0, train accr: 0.05601673662612147, val accr: 0.07817403708987161

Epoch: 0, train accr tags: 0.36281464991215956, val accr tags: 0.4117689015691869


  0%|          | 0/400 [00:00<?, ?it/s]


Epoch: 1, train loss: 7.297870186567306, val loss: 7.279327535629273

Epoch: 1, train accr: 0.10737599240008143, val accr: 0.10968498897816967

Epoch: 1, train accr tags: 0.44399810002035695, val accr tags: 0.43809997866742517


  0%|          | 0/400 [00:00<?, ?it/s]

...


Epoch: 17, train loss: 3.865159631371498, val loss: 7.12977782189846

Epoch: 17, train accr: 0.39209714154795755, val accr: 0.15765955949616495

Epoch: 17, train accr tags: 0.5127963234996833, val accr tags: 0.4556681443951868


  0%|          | 0/400 [00:00<?, ?it/s]


Epoch: 18, train loss: 3.735274168848991, val loss: 7.143848976492881

Epoch: 18, train accr: 0.40771895101434935, val accr: 0.16508168739331872

Epoch: 18, train accr tags: 0.5195715260039852, val accr tags: 0.46459051799212736


100%|██████████| 400/400 [03:30<00:00,  1.12it/s]



Epoch: 19, train loss: 3.6728498983383178, val loss: 7.258537444472313

Epoch: 19, train accr: 0.42210088893072795, val accr: 0.15643330672312658

Epoch: 19, train accr tags: 0.523487405693998, val accr tags: 0.463989448474541
Finished!


In [77]:
torch.save(network.state_dict(), 'add_pos_tagging_inner_layer.pwf')

In [78]:
on_train, on_train_tags, all_accr, all_accr_tags = try_lengthes(train_lem)
on_train[:10], on_train_tags[:10], all_accr, all_accr_tags

(array([0.22645311, 0.3594081 , 0.50398971, 0.55208333, 0.59709455,
        0.60381384, 0.62298748, 0.62519026, 0.60157791, 0.6155633 ]),
 array([0.61431155, 0.53768534, 0.51703664, 0.49819303, 0.4562541 ,
        0.43232431, 0.42620751, 0.42656012, 0.41485865, 0.42508711]),
 0.4249589866336767,
 0.528104241185488)

In [79]:
on_test, on_test_tags, all_accr, all_accr_tags = try_lengthes(test_lem)
on_test[:10], on_test_tags[:10], all_accr, all_accr_tags

(array([0.13437086, 0.14207198, 0.1718036 , 0.18258618, 0.18599428,
        0.20033389, 0.17665953, 0.17244526, 0.14785374, 0.09221902]),
 array([0.58384106, 0.48004819, 0.4305299 , 0.41122769, 0.38321764,
        0.36093489, 0.32762313, 0.35766423, 0.34022258, 0.32276657]),
 0.15766613085262834,
 0.46352474434830193)